In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from google.colab import drive
import xarray as xr
import gdal
import os
import os.path
from os import path

In [ ]:
drive.mount('/content/drive')

In [ ]:
!pip install rioxarray

In [ ]:
import rioxarray as rio

# **1. Convert NetCDF to Tiff**

In [ ]:
# Create Directory on google Drive
for f in ['albedo_01', 'albedo_02', 'albedo_03', 'albedo_04', 'albedo_05', 'albedo_06', 'tbb_07', 'tbb_08', 'tbb_09', 'tbb_10', 'tbb_11', 'tbb_12', 'tbb_13', 'tbb_14', 'tbb_15', 'tbb_16']:
    for d in ['01', '02', '03', '04', '05', '08', '09', '10', '15', '16', '17', '18', '23', '24', '25', '28', '29', '30', '31']:
        if path.exists('/content/drive/My Drive/Rainfall Data/Tiff/' + str(f)) == False:
            os.mkdir('/content/drive/My Drive/Rainfall Data/Tiff/' + str(f))
        if path.exists('/content/drive/My Drive/Rainfall Data/Tiff/' + str(f) + '/' + str(d)) == False:
            os.mkdir('/content/drive/My Drive/Rainfall Data/Tiff/' + str(f) + '/' + str(d))

In [ ]:
# Function to call Gdalwarp and some options to clipping the raster file after extract from netCDF
def clipping_raster(src_file, dest_file):
    ds = gdal.Warp(srcDSOrSrcDSTab= src_file \
          , destNameOrDestDS = dest_file \
          ,warpOptions = ['multithread = True', 'CUTLINE_ALL_TOUCHED=TRUE' ] \
          ,cutlineDSName="/content/drive/My Drive/Rainfall Data/Ha_Tinh_SHP/Ha_Tinh.shp" \
          , cropToCutline=True)
    if ds is None:
        return 'fail'
    ds = None
    return 'success'

# Read files netCDF and extract each band in order to clipping around Ha Tinh province with shape file of Ha Tinh.
def convert_nc_to_Tiff():
    # Handle number of errors
    num_err = 0
    for day in range(31):
        # if day == 16: continue
        path = '/content/drive/My Drive/Rainfall Data/CDF/' + str(day).rjust(2, '0')
        # Check if directories exist or not
        if os.path.isdir(path) == False:
            continue
        path = path + '/NC_H08_201908' + str(day).rjust(2, '0') + '_'

        for hour in range(24):
            for mins in [0, 10, 20, 30, 40, 50]:
                fpath = path + str(hour).rjust(2, '0') + str(mins).rjust(2, '0') + '_R21_FLDK.02401_02401.nc'
                # Check if files exist or not
                if os.path.isfile(fpath) == False: continue
                nc_file = xr.open_dataset(fpath)
                
                # Extract each band of Metadata
                for b in ['albedo_01', 'albedo_02', 'albedo_03', 'albedo_04', 'albedo_05', 'albedo_06', \
                          'tbb_07', 'tbb_08', 'tbb_09', 'tbb_10', 'tbb_11', 'tbb_12', 'tbb_13', 'tbb_14', 'tbb_15', 'tbb_16']:
                    
                    fname = 'H08_' + b + '_201908' + str(day).rjust(2, '0') +  '_' + str(hour).rjust(2, '0') + str(mins).rjust(2, '0') + '.tiff'
                    src_file = "/content/drive/My Drive/Rainfall Data/Tiff/material/" + fname
                    dest_file = "/content/drive/My Drive/Rainfall Data/Tiff/" + b + "/" + str(day).rjust(2, '0') + "/" + fname

                    # For continue Running clip along discrete time running this executive function
                    if (os.path.isfile(src_file) == True) and (os.path.isfile(dest_file) == True): continue

                    band = nc_file[b]
                    band = band.rio.set_spatial_dims('longitude', 'latitude')
                    band = band.rio.set_crs("epsg:4326")
                    band.rio.to_raster(src_file)

                    # Execute clipping Function
                    if clipping_raster(src_file, dest_file) == 'fail':
                        num_err += 1

    return num_err

In [ ]:
# Execute Function and print out number of errors
num_errors = convert_nc_to_Tiff()
num_errors

0

**Đã thực hiện thành công toàn bộ việc clip và save lại các file Tiff với 0 error**

Bài toán phân hoạch trong Probability and Statistics:

Cho $(a + b)$ tấm thẻ trong đó có $a$ tấm thẻ Đỏ và $b$ tấm thẻ Xanh.

Có $(a + b)$ hộp trong đó có $a$ hộp Đỏ và $b$ hộp Xanh.

Sắp ngẫu nhiên $(a + b)$ tấm thẻ vào $(a + b)$ hộp, gọi $X$ là biến ngẫu nhiên của Số hộp và thẻ được xếp cùng màu. $P(x = X) = ?$

Nhận xét: Không gian mẫu của bài toán sẽ là phân hoạch cả số thẻ và số hộp với 2 nhóm trùng lặp là Đỏ và Xanh $\implies \begin{align} \Omega = \bigg(\frac{(a + b)!}{a!b!}\bigg)^2 \end{align}$

Không mất tính tổng quát, nếu $a < b$, ta thực hiện $Swap(a, b)$. Xét trường hợp giả thiết $a > b$. Khi ta cố tình sắp ra một số cặp hộp - thẻ là Đỏ hoặc Xanh để gán cho X thì số thẻ Đỏ còn lại phải bằng số hộp Xanh và số thẻ Xanh còn lại phải bằng số hộp Đỏ để thỏa mãn tính chất không trùng màu đối nghịch với tính chất của $X$.

$\implies$ Giá trị đầu tiên của $X$ trong bảng phân phối sẽ là $(a - b)$. Những giá trị tiếp theo cũng phải đảm bảo điều kiện số thẻ Đỏ còn lại phải bằng số hộp Xanh và số thẻ Xanh còn lại phải bằng số hộp Đỏ nên mỗi khi ta lấy 1 cặp hộp - thẻ màu Đỏ ra khỏi nhóm không trùng màu để tăng X lên thì la phải lấy thêm 1 cặp hộp - thẻ màu Xanh. $\implies$ Ở mỗi cột tiếp theo của bảng phân phối sẽ là $(a - b) + 2k \implies X$ sẽ có closed form là $X_i = (a - b) + 2i$ với $i \in [0, b]$. Hơn thế nữa, ta còn chỉ ra được cụ thể sẽ có bao nhiêu cặp hộp - thẻ trùng màu Đỏ hay Xanh đóng góp vào trọng số của $X$. Tại $X_0$, số cặp hộp - thẻ trùng sẽ luôn là $(a - b)$ Đỏ, nói cách khác khi ta khởi tạo $X_0$ là ta đang khử lượng dư của thẻ Đỏ so với hộp Xanh.

Tại $X_1$ ta sẽ có thêm 1 cặp hộp - thẻ Đỏ và 1 cặp hộp - thẻ Xanh, hiểu theo 1 nghĩa khác ta sẽ phải khử lượng dư để thỏa mãn cả 2 ràng buộc số thẻ Xanh $=$ số hộp Đỏ và số thẻ Đỏ $=$ số hộp Xanh. $\implies$ ta có $(a - b) + 1$ cặp hộp - thẻ Đỏ và $1$ cặp hộp - thẻ Xanh.

Tổng quát hóa, Tại $X_i$ ta sẽ có $(a - b) + i$ cặp hộp - thẻ Đỏ và $i$ cặp hộp - thẻ Xanh, mặt khác ta còn có $2(b - i)$ cặp hộp - thẻ không trùng màu. Việc chỉ rõ số lượng các loại hộp - thẻ trùng nhau và số cặp hộp - thẻ không trùng màu sẽ làm sáng tỏ công thức khi ta thực hiện phân hoạch.

Chia số các cặp hộp - thẻ ra làm đôi, 1 bên chỉ có các hộp - thẻ trùng màu, một bên là các cặp không trùng màu. Ta sẽ thực hiện phân hoạch trên đoạn các cặp không trùng màu, sau đó thực hiện phân hoạch trộn các cặp trùng màu vào các cặp không trùng màu.

Tại $X_i$: Trong các cặp không trùng màu, ta chỉ cần phân hoạch trên dãy thẻ hoặc dãy hộp vì cách xếp một dãy là song ánh của cách xếp dãy còn lại.

Xét cách phân hoạch dãy thẻ: $(b - i)$ thẻ Đỏ và Xanh mỗi loại $\implies$ Có $\begin{align} \frac{(2b - 2i)!}{(b - i)!(b - i)!}\end{align}$ cách xếp.

Giờ ta sẽ tìm cách phân hoạch trộn các cặp trùng màu vào các cặp không trùng màu này: $(a - b + i)$ cặp trùng Đỏ, $i$ cặp trùng Xanh, $(2b - 2i)$ cặp không trùng được phân hoặc trộn trên tổng $(a + b)$ cặp.

$\implies$ Sẽ có $\begin{align} \frac{(a + b)!}{(a - b + i)! \times i! \times(2b - 2i)!}\end{align}$ cách xếp.

Vì việc phân hoạch dãy không trùng và phân hoặc trộn 2 dãy diễn ra độc lập nên tổng số cách xếp để có $X_i$ cặp trùng màu hộp - thẻ là:
$\implies$ Sẽ có $\begin{align} \frac{(a + b)!}{(a - b + i)! \times i! \times(2b - 2i)!} \times \frac{(2b - 2i)!}{(b - i)!(b - i)!} = \frac{(a + b)!}{(a - b + i)! \times i! \times(b - i)! \times(b - i)!}\end{align}$ cách xếp.

$\begin{align} \implies P(X = X_i) = \frac{(a + b)!}{(a - b + i)! \times i! \times(b - i)! \times(b - i)!} \div  \bigg(\frac{(a + b)!}{a!b!}\bigg)^2  = \frac{(a!b!)^2}{(a + b)! \times (a - b + i)! \times i! \times\big((b - i)!\big)^2}\end{align}$

---

Solution của bài toán với $a = 4, b = 3$: 4 thẻ Đỏ, 3 thẻ Xanh, 4 hộp Đỏ, 3 hộp Xanh.

Không gian mẫu của bài toán theo công thức sẽ là $35^2$

Nhận thấy rằng $X$ sẽ chỉ nhận các giá trị $1, 3, 5, 7$.

- $\begin{align} P(X = X_0 = 1) = \frac{(4!3!)^2}{7! \times 1! \times 0! \times\big(3!\big)^2} = \frac{4}{35}\end{align}$
- $\begin{align} P(X = X_1 = 3) = \frac{(4!3!)^2}{7! \times 2! \times 1! \times\big(2!\big)^2} = \frac{18}{35}\end{align}$
- $\begin{align} P(X = X_2 = 5) = \frac{(4!3!)^2}{7! \times 3! \times 2! \times\big(1!\big)^2} = \frac{12}{35}\end{align}$
- $\begin{align} P(X = X_3 = 7) = \frac{(4!3!)^2}{7! \times 4! \times 3! \times\big(0!\big)^2} = \frac{1}{35}\end{align}$

$\implies$ Bảng phân bố của BNN $X$:

<table>
<tr>
<td>$\begin{align}X\end{align}$</td>
<td>$\begin{align}1\end{align}$</td>
<td>$\begin{align}3\end{align}$</td>
<td>$\begin{align}5\end{align}$</td>
<td>$\begin{align}7\end{align}$</td>
</tr>
<tr>
<td>$\begin{align}\end{align}$</td>
<td>$\begin{align}\frac{4}{35}\end{align}$</td>
<td>$\begin{align}\frac{18}{35}\end{align}$</td>
<td>$\begin{align}\frac{12}{35}\end{align}$</td>
<td>$\begin{align}\frac{1}{35}\end{align}$</td>
</tr>
</table>